In [1]:
# You're given a results_nlp.json of the format below:
# {
#   "stage-final-llava-v15-phi1_5+1b": {
#     "vqa-v2": 0.0,
#     "textvqa-ocr": 0.0,
#     "textvqa-pure": 0.0,
#     "gqa": 0.0,
#     "refcoco": 0.0,
#     "wsc273": 0.8095238095238095,
#     "winogrande": 0.7245461720599842,
#     "lambada_standard": 0.3114690471569959,
#     "arc_easy": 0.742003367003367,
#     "arc_challenge": 0.40102389078498296
#   },
#   "stage-final-llava-v15-pythia+1p4b-sgm-old": {
#     "vqa-v2": 0.5317000000000001,
#     "textvqa-ocr": 0.28222656250000006,
#     "textvqa-pure": 0.23583984375000014,
#     "gqa": 0.36619999999999997,
#     "refcoco": 0.0234375,
#     "wsc273": 0.717948717948718,
#     "winogrande": 0.5737963693764798,
#     "lambada_standard": 0.4513875412381137,
#     "arc_easy": 0.6014309764309764,
#     "arc_challenge": 0.2841296928327645
#   },
#   "stage-final-llava-v15-pythia+160m-ptune": {
#     "vqa-v2": 0.0,
#     "textvqa-ocr": 0.0,
#     "textvqa-pure": 0.0,
#     "gqa": 0.0,
#     "refcoco": 0.0
#   },
# }

# We are concerned with the following models:
# pythia+1p4b and pythia+1p4b-instruct: Respectively:
# 	reproduction-align-pythia+1p4b and reproduction-align-pythia+1p4b-instruct for their language only LLMs method
# 	stage-final-llava-v15-pythia+1p4b and stage-final-llava-v15-pythia+1p4b-instruct for their Original LLaVA method
# LLaMA2 7B and Vicuna1.5 7B:
	
# 	and reproduction-llava-v15+7b+stage-finetune+x7 for the Original LLaVA method


import os
import json
import numpy as np

# Define the datasets of interest
datasets = [
    'vqa-v2', 'textvqa-ocr', 'textvqa-pure', 'gqa', 'refcoco',
    'wsc273', 'winogrande', 'lambada_standard', 'arc_easy', 'arc_challenge'
]

# Define the result dictionary
result = {}

# Path to the JSON results file
results_file = 'results_nlp.json'

# Read the JSON results file
if os.path.isfile(results_file):
    with open(results_file, 'r') as f:
        result = json.load(f)
else:
    print(f"Error: File {results_file} not found.")


Top 3 only

import json
import numpy as np

# Calculate delta values
original_llava_acc = results_dict["Original LLaVA"]
language_only_llm_acc = results_dict.get("Language Only LLM", {})

table_data = []

for model, accuracies in results_dict.items():
    if model in ["Original LLaVA", "Language Only LLM"]:
        continue
    deltas = {dataset: acc - original_llava_acc[dataset] for dataset, acc in accuracies.items()}
    avg_delta_vl = sum(deltas[dataset] for dataset in accuracies if dataset in ["vqa-v2", "textvqa-ocr", "textvqa-pure", "gqa"]) / 4
    avg_acc_vl = sum(accuracies[dataset] for dataset in accuracies if dataset in ["vqa-v2", "textvqa-ocr", "textvqa-pure", "gqa"]) / 4
    avg_delta_nlu = sum(deltas[dataset] for dataset in accuracies if dataset in ["wsc273", "winogrande", "arc_easy", "arc_challenge", "lambada_standard"]) / 5
    avg_acc_nlu = sum(accuracies[dataset] for dataset in accuracies if dataset in ["wsc273", "winogrande", "arc_easy", "arc_challenge", "lambada_standard"]) / 5
    table_data.append((model, accuracies, deltas, avg_delta_vl, avg_acc_vl, avg_delta_nlu, avg_acc_nlu))

# Sort the data so models with smallest average delta VL are first
table_data = sorted(table_data, key=lambda x: x[3])

# Function to format values or return "-"
def format_value(value):
    return "{:.1f}".format(value * 100) if not np.isnan(value) else "-"

# Generate LaTeX table for Vision-Language part
latex_code_vl = """
\\begin{table*}[h]
  \\caption{\\textbf{LLaVA Model Performance:} Vision-Language (VL) Tasks}
  \\label{tab:vl_acc}
  \\centering
  \\resizebox{\\linewidth}{!}{
    \\begin{tabular}{l|cccc|cc}
     \\toprule
     \\textbf{Model} & \\multicolumn{4}{c|}{\\textbf{Vision-Language (VL)}} & \\multicolumn{2}{c}{\\textbf{VL Avg.}} \\\\
     & \\textbf{VQAv2} & \\textbf{TextVQA OCR} & \\textbf{TextVQA Pure} & \\textbf{GQA} & $\\Delta$ & Acc \\\\
     \\midrule
"""

latex_code_vl += "Original LLaVA & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & - & {avg_acc_vl} \\\\\n".format(
    vqa_v2=format_value(original_llava_acc["vqa-v2"]),
    textvqa_ocr=format_value(original_llava_acc["textvqa-ocr"]),
    textvqa_pure=format_value(original_llava_acc["textvqa-pure"]),
    gqa=format_value(original_llava_acc["gqa"]),
    avg_acc_vl=format_value((original_llava_acc["vqa-v2"] + original_llava_acc["textvqa-ocr"] + original_llava_acc["textvqa-pure"] + original_llava_acc["gqa"]) / 4)
)

latex_code_vl += "Language Only LLM & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & - & {avg_acc_vl} \\\\\n".format(
    vqa_v2=format_value(language_only_llm_acc.get("vqa-v2", np.nan)),
    textvqa_ocr=format_value(language_only_llm_acc.get("textvqa-ocr", np.nan)),
    textvqa_pure=format_value(language_only_llm_acc.get("textvqa-pure", np.nan)),
    gqa=format_value(language_only_llm_acc.get("gqa", np.nan)),
    avg_acc_vl=format_value((language_only_llm_acc.get("vqa-v2", 0) + language_only_llm_acc.get("textvqa-ocr", 0) + language_only_llm_acc.get("textvqa-pure", 0) + language_only_llm_acc.get("gqa", 0)) / 4)
)

latex_code_vl += "\\midrule\n"

for model, accuracies, deltas, avg_delta_vl, avg_acc_vl, _, _ in table_data:
    latex_code_vl += "{model} & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {delta_vl} & {acc_vl} \\\\\n".format(
        model=model,
        vqa_v2=format_value(accuracies["vqa-v2"]),
        textvqa_ocr=format_value(accuracies["textvqa-ocr"]),
        textvqa_pure=format_value(accuracies["textvqa-pure"]),
        gqa=format_value(accuracies["gqa"]),
        delta_vl=format_value(avg_delta_vl),
        acc_vl=format_value(avg_acc_vl)
    )

latex_code_vl += """
     \\bottomrule
    \\end{tabular}
  }
\\end{table*}
"""

print(latex_code_vl)

# Generate LaTeX table for NLU/NLG part
latex_code_nlu = """
\\begin{table*}[h]
  \\caption{\\textbf{LLaVA Model Performance:} NLU/NLG Tasks}
  \\label{tab:nlu_acc}
  \\centering
  \\resizebox{\\linewidth}{!}{
    \\begin{tabular}{l|ccccc|cc}
     \\toprule
     \\textbf{Model} & \\multicolumn{5}{c|}{\\textbf{NLU/NLG}} & \\multicolumn{2}{c}{\\textbf{NLU Avg.}} \\\\
     & \\textbf{WSC273} & \\textbf{Winogrande} & \\textbf{ARC Easy} & \\textbf{ARC Challenge} & \\textbf{Lambada} & $\\Delta$ & Acc \\\\
     \\midrule
"""

latex_code_nlu += "Original LLaVA & {wsc273} & {winogrande} & {arc_easy} & {arc_challenge} & {lambada} & - & {avg_acc_nlu} \\\\\n".format(
    wsc273=format_value(original_llava_acc["wsc273"]),
    winogrande=format_value(original_llava_acc["winogrande"]),
    arc_easy=format_value(original_llava_acc["arc_easy"]),
    arc_challenge=format_value(original_llava_acc["arc_challenge"]),
    lambada=format_value(original_llava_acc["lambada_standard"]),
    avg_acc_nlu=format_value((original_llava_acc["wsc273"] + original_llava_acc["winogrande"] + original_llava_acc["arc_easy"] + original_llava_acc["arc_challenge"] + original_llava_acc["lambada_standard"]) / 5)
)

latex_code_nlu += "Language Only LLM & {wsc273} & {winogrande} & {arc_easy} & {arc_challenge} & {lambada} & - & {avg_acc_nlu} \\\\\n".format(
    wsc273=format_value(language_only_llm_acc.get("wsc273", np.nan)),
    winogrande=format_value(language_only_llm_acc.get("winogrande", np.nan)),
    arc_easy=format_value(language_only_llm_acc.get("arc_easy", np.nan)),
    arc_challenge=format_value(language_only_llm_acc.get("arc_challenge", np.nan)),
    lambada=format_value(language_only_llm_acc.get("lambada_standard", np.nan)),
    avg_acc_nlu=format_value((language_only_llm_acc.get("wsc273", 0) + language_only_llm_acc.get("winogrande", 0) + language_only_llm_acc.get("arc_easy", 0) + language_only_llm_acc.get("arc_challenge", 0) + language_only_llm_acc.get("lambada_standard", 0)) / 5)
)

latex_code_nlu += "\\midrule\n"

for model, accuracies, deltas, _, _, avg_delta_nlu, avg_acc_nlu in table_data:
    latex_code_nlu += "{model} & {wsc273} & {winogrande} & {arc_easy} & {arc_challenge} & {lambada} & {delta_nlu} & {acc_nlu} \\\\\n".format(
        model=model,
        wsc273=format_value(accuracies["wsc273"]),
        winogrande=format_value(accuracies["winogrande"]),
        arc_easy=format_value(accuracies["arc_easy"]),
        arc_challenge=format_value(accuracies["arc_challenge"]),
        lambada=format_value(accuracies["lambada_standard"]),
        delta_nlu=format_value(avg_delta_nlu),
        acc_nlu=format_value(avg_acc_nlu)
    )

latex_code_nlu += """
     \\bottomrule
    \\end{tabular}
  }
\\end{table*}
"""

print(latex_code_nlu)


In [14]:
import json
import numpy as np

# Calculate delta values
original_llava_acc = results_dict["Original LLaVA"]
language_only_llm_acc = results_dict.get("Language Only LLM", {})

table_data = []

for model, accuracies in results_dict.items():
    if model in ["Original LLaVA", "Language Only LLM"]:
        continue
    avg_acc_vl = sum(accuracies[dataset] for dataset in accuracies if dataset in ["vqa-v2", "textvqa-ocr", "textvqa-pure", "gqa"]) / 4
    nlu_deltas = {dataset: accuracies[dataset] - language_only_llm_acc.get(dataset, 0) for dataset in ["wsc273", "winogrande", "arc_easy", "arc_challenge"]}
    avg_delta_nlu = sum(nlu_deltas[dataset] for dataset in nlu_deltas) / 4
    avg_acc_nlu = sum(accuracies[dataset] for dataset in nlu_deltas) / 4
    delta_nlg = accuracies["lambada_standard"] - language_only_llm_acc.get("lambada_standard", 0)
    avg_acc_nlg = accuracies["lambada_standard"]
    table_data.append((model, accuracies, avg_acc_vl, avg_delta_nlu, avg_acc_nlu, delta_nlg, avg_acc_nlg))

# Sort the data by Avg. VL Accuracy and highest NLG Delta
table_data = sorted(table_data, key=lambda x: (x[2], -x[5]), reverse=True)

# Separate the data into two groups
lora_variants = [
    'LoRA (1/4 Full Rank)', 'LoRA (1/4 Full Rank, Higher Alpha)', 
    'LoRA (1/2 Full Rank)', 'LoRA (1/2 Full Rank, Higher Alpha)', 
    'LoRA (1/2 Full Rank, RSLoRA)', 'LoRA (1/2 Full Rank, RSLoRA, KQV Target)'
]

lora_table_data = [item for item in table_data if item[0] in lora_variants]
other_table_data = [item for item in table_data if item[0] not in lora_variants]

# Function to format values or return "-"
def format_value(value):
    return "{:.1f}".format(value * 100) if not np.isnan(value) else "-"

# Generate LaTeX table for LoRA variants
latex_code_lora = """
\\begin{table*}[h]
  \\caption{\\textbf{LLaVA Model Performance:} LoRA Variants}
  \\label{tab:lora_variants_acc}
  \\centering
  \\resizebox{\\linewidth}{!}{
    \\begin{tabular}{l|cccc|c|cc|cc}
     \\toprule
     \\textbf{Model} & \\multicolumn{4}{c|}{\\textbf{Vision-Language (VL)}} & \\textbf{VL Avg.} & \\multicolumn{2}{c|}{\\textbf{NLU Avg.}} & \\multicolumn{2}{c}{\\textbf{NLG Avg.}} \\\\
     & \\textbf{VQAv2} & \\textbf{TextVQA OCR} & \\textbf{TextVQA Pure} & \\textbf{GQA} & Acc $\\uparrow$ & $\\Delta \\uparrow$ & Acc $\\uparrow$ & $\\Delta \\uparrow$ & Acc $\\uparrow$ \\\\
     \\midrule
"""

original_llava_deltas = {dataset: original_llava_acc[dataset] - language_only_llm_acc.get(dataset, 0) for dataset in ["wsc273", "winogrande", "arc_easy", "arc_challenge"]}
original_llava_avg_delta_nlu = sum(original_llava_deltas[dataset] for dataset in original_llava_deltas) / 4
original_llava_delta_nlg = original_llava_acc["lambada_standard"] - language_only_llm_acc.get("lambada_standard", 0)

latex_code_lora += "Original LLaVA & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {avg_acc_vl} & {delta_nlu} & {avg_acc_nlu} & {delta_nlg} & {avg_acc_nlg} \\\\\n".format(
    vqa_v2=format_value(original_llava_acc["vqa-v2"]),
    textvqa_ocr=format_value(original_llava_acc["textvqa-ocr"]),
    textvqa_pure=format_value(original_llava_acc["textvqa-pure"]),
    gqa=format_value(original_llava_acc["gqa"]),
    avg_acc_vl=format_value((original_llava_acc["vqa-v2"] + original_llava_acc["textvqa-ocr"] + original_llava_acc["textvqa-pure"] + original_llava_acc["gqa"]) / 4),
    delta_nlu=format_value(original_llava_avg_delta_nlu),
    avg_acc_nlu=format_value((original_llava_acc["wsc273"] + original_llava_acc["winogrande"] + original_llava_acc["arc_easy"] + original_llava_acc["arc_challenge"]) / 4),
    delta_nlg=format_value(original_llava_delta_nlg),
    avg_acc_nlg=format_value(original_llava_acc["lambada_standard"])
)

latex_code_lora += "Language Only LLM & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {avg_acc_vl} & - & {avg_acc_nlu} & - & {avg_acc_nlg} \\\\\n".format(
    vqa_v2=format_value(language_only_llm_acc.get("vqa-v2", np.nan)),
    textvqa_ocr=format_value(language_only_llm_acc.get("textvqa-ocr", np.nan)),
    textvqa_pure=format_value(language_only_llm_acc.get("textvqa-pure", np.nan)),
    gqa=format_value(language_only_llm_acc.get("gqa", np.nan)),
    avg_acc_vl=format_value((language_only_llm_acc.get("vqa-v2", 0) + language_only_llm_acc.get("textvqa-ocr", 0) + language_only_llm_acc.get("textvqa-pure", 0) + language_only_llm_acc.get("gqa", 0)) / 4),
    avg_acc_nlu=format_value((language_only_llm_acc.get("wsc273", 0) + language_only_llm_acc.get("winogrande", 0) + language_only_llm_acc.get("arc_easy", 0) + language_only_llm_acc.get("arc_challenge", 0)) / 4),
    avg_acc_nlg=format_value(language_only_llm_acc.get("lambada_standard", np.nan))
)

latex_code_lora += "\\midrule\n"

for model, accuracies, avg_acc_vl, avg_delta_nlu, avg_acc_nlu, delta_nlg, avg_acc_nlg in lora_table_data:
    latex_code_lora += "{model} & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {acc_vl} & {delta_nlu} & {acc_nlu} & {delta_nlg} & {acc_nlg} \\\\\n".format(
        model=model,
        vqa_v2=format_value(accuracies["vqa-v2"]),
        textvqa_ocr=format_value(accuracies["textvqa-ocr"]),
        textvqa_pure=format_value(accuracies["textvqa-pure"]),
        gqa=format_value(accuracies["gqa"]),
        acc_vl=format_value(avg_acc_vl),
        delta_nlu=format_value(avg_delta_nlu),
        acc_nlu=format_value(avg_acc_nlu),
        delta_nlg=format_value(delta_nlg),
        acc_nlg=format_value(avg_acc_nlg)
    )

latex_code_lora += """
     \\bottomrule
    \\end{tabular}
  }
\\end{table*}
"""

# Generate LaTeX table for other methods
latex_code_other = """
\\begin{table*}[h]
  \\caption{\\textbf{LLaVA Model Performance:} Other Methods}
  \\label{tab:other_methods_acc}
  \\centering
  \\resizebox{\\linewidth}{!}{
    \\begin{tabular}{l|cccc|c|cc|cc}
     \\toprule
     \\textbf{Model} & \\multicolumn{4}{c|}{\\textbf{Vision-Language (VL)}} & \\textbf{VL Avg.} & \\multicolumn{2}{c|}{\\textbf{NLU Avg.}} & \\multicolumn{2}{c}{\\textbf{NLG Avg.}} \\\\
     & \\textbf{VQAv2} & \\textbf{TextVQA OCR} & \\textbf{TextVQA Pure} & \\textbf{GQA} & Acc $\\uparrow$ & $\\Delta \\uparrow$ & Acc $\\uparrow$ & $\\Delta \\uparrow$ & Acc $\\uparrow$ \\\\
     \\midrule
"""

latex_code_other += "Original LLaVA & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {avg_acc_vl} & {delta_nlu} & {avg_acc_nlu} & {delta_nlg} & {avg_acc_nlg} \\\\\n".format(
    vqa_v2=format_value(original_llava_acc["vqa-v2"]),
    textvqa_ocr=format_value(original_llava_acc["textvqa-ocr"]),
    textvqa_pure=format_value(original_llava_acc["textvqa-pure"]),
    gqa=format_value(original_llava_acc["gqa"]),
    avg_acc_vl=format_value((original_llava_acc["vqa-v2"] + original_llava_acc["textvqa-ocr"] + original_llava_acc["textvqa-pure"] + original_llava_acc["gqa"]) / 4),
    delta_nlu=format_value(original_llava_avg_delta_nlu),
    avg_acc_nlu=format_value((original_llava_acc["wsc273"] + original_llava_acc["winogrande"] + original_llava_acc["arc_easy"] + original_llava_acc["arc_challenge"]) / 4),
    delta_nlg=format_value(original_llava_delta_nlg),
    avg_acc_nlg=format_value(original_llava_acc["lambada_standard"])
)

latex_code_other += "Language Only LLM & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {avg_acc_vl} & - & {avg_acc_nlu} & - & {avg_acc_nlg} \\\\\n".format(
    vqa_v2=format_value(language_only_llm_acc.get("vqa-v2", np.nan)),
    textvqa_ocr=format_value(language_only_llm_acc.get("textvqa-ocr", np.nan)),
    textvqa_pure=format_value(language_only_llm_acc.get("textvqa-pure", np.nan)),
    gqa=format_value(language_only_llm_acc.get("gqa", np.nan)),
    avg_acc_vl=format_value((language_only_llm_acc.get("vqa-v2", 0) + language_only_llm_acc.get("textvqa-ocr", 0) + language_only_llm_acc.get("textvqa-pure", 0) + language_only_llm_acc.get("gqa", 0)) / 4),
    avg_acc_nlu=format_value((language_only_llm_acc.get("wsc273", 0) + language_only_llm_acc.get("winogrande", 0) + language_only_llm_acc.get("arc_easy", 0) + language_only_llm_acc.get("arc_challenge", 0)) / 4),
    avg_acc_nlg=format_value(language_only_llm_acc.get("lambada_standard", np.nan))
)

latex_code_other += "\\midrule\n"

for model, accuracies, avg_acc_vl, avg_delta_nlu, avg_acc_nlu, delta_nlg, avg_acc_nlg in other_table_data:
    latex_code_other += "{model} & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {acc_vl} & {delta_nlu} & {acc_nlu} & {delta_nlg} & {acc_nlg} \\\\\n".format(
        model=model,
        vqa_v2=format_value(accuracies["vqa-v2"]),
        textvqa_ocr=format_value(accuracies["textvqa-ocr"]),
        textvqa_pure=format_value(accuracies["textvqa-pure"]),
        gqa=format_value(accuracies["gqa"]),
        acc_vl=format_value(avg_acc_vl),
        delta_nlu=format_value(avg_delta_nlu),
        acc_nlu=format_value(avg_acc_nlu),
        delta_nlg=format_value(delta_nlg),
        acc_nlg=format_value(avg_acc_nlg)
    )

latex_code_other += """
     \\bottomrule
    \\end{tabular}
  }
\\end{table*}
"""

print(latex_code_lora)
print(latex_code_other)



\begin{table*}[h]
  \caption{\textbf{LLaVA Model Performance:} LoRA Variants}
  \label{tab:lora_variants_acc}
  \centering
  \resizebox{\linewidth}{!}{
    \begin{tabular}{l|cccc|c|cc|cc}
     \toprule
     \textbf{Model} & \multicolumn{4}{c|}{\textbf{Vision-Language (VL)}} & \textbf{VL Avg.} & \multicolumn{2}{c|}{\textbf{NLU Avg.}} & \multicolumn{2}{c}{\textbf{NLG Avg.}} \\
     & \textbf{VQAv2} & \textbf{TextVQA OCR} & \textbf{TextVQA Pure} & \textbf{GQA} & Acc $\uparrow$ & $\Delta \uparrow$ & Acc $\uparrow$ & $\Delta \uparrow$ & Acc $\uparrow$ \\
     \midrule
Original LLaVA & 30.3 & 2.4 & 3.8 & 22.2 & 14.7 & -1.5 & 41.1 & -12.0 & 11.3 \\
Language Only LLM & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & - & 42.7 & - & 23.3 \\
\midrule
LoRA (1/2 Full Rank, Higher Alpha) & 28.7 & 1.1 & 2.7 & 19.7 & 13.0 & -1.0 & 41.6 & -13.2 & 10.1 \\
LoRA (1/2 Full Rank, RSLoRA) & 29.0 & 1.0 & 1.7 & 18.0 & 12.4 & -1.5 & 41.2 & -3.9 & 19.5 \\
LoRA (1/4 Full Rank) & 24.6 & 0.9 & 1.4 & 15.0 & 10.5 & -2.6 & 40.0 & -15.

In [11]:
import json
import numpy as np

# Calculate delta values
original_llava_acc = results_dict["Original LLaVA"]
language_only_llm_acc = results_dict.get("Language Only LLM", {})

table_data = []

for model, accuracies in results_dict.items():
    if model not in ["Original LLaVA", "Language Only LLM"]:
        table_data.append((model, accuracies))

# Sort the data by model name
table_data = sorted(table_data, key=lambda x: x[0])

# Function to format values or return "-"
def format_value(value):
    return "{:.1f}".format(value * 100) if not np.isnan(value) else "-"

# Generate LaTeX table
latex_code = """
\\begin{table*}[h]
  \\caption{\\textbf{LLaVA Model Performance:} Per Dataset}
  \\label{tab:per_dataset}
  \\centering
  \\resizebox{\\linewidth}{!}{
    \\begin{tabular}{l|cccccccccc}
     \\toprule
     \\textbf{Model} & \\textbf{VQAv2} & \\textbf{TextVQA OCR} & \\textbf{TextVQA Pure} & \\textbf{GQA} & \\textbf{WSC273} & \\textbf{Winogrande} & \\textbf{ARC Easy} & \\textbf{ARC Challenge} & \\textbf{Lambada} \\\\
     \\midrule
"""

latex_code += "Original LLaVA & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {wsc273} & {winogrande} & {arc_easy} & {arc_challenge} & {lambada} \\\\\n".format(
    vqa_v2=format_value(original_llava_acc["vqa-v2"]),
    textvqa_ocr=format_value(original_llava_acc["textvqa-ocr"]),
    textvqa_pure=format_value(original_llava_acc["textvqa-pure"]),
    gqa=format_value(original_llava_acc["gqa"]),
    wsc273=format_value(original_llava_acc["wsc273"]),
    winogrande=format_value(original_llava_acc["winogrande"]),
    arc_easy=format_value(original_llava_acc["arc_easy"]),
    arc_challenge=format_value(original_llava_acc["arc_challenge"]),
    lambada=format_value(original_llava_acc["lambada_standard"])
)

latex_code += "Language Only LLM & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {wsc273} & {winogrande} & {arc_easy} & {arc_challenge} & {lambada} \\\\\n".format(
    vqa_v2=format_value(language_only_llm_acc.get("vqa-v2", np.nan)),
    textvqa_ocr=format_value(language_only_llm_acc.get("textvqa-ocr", np.nan)),
    textvqa_pure=format_value(language_only_llm_acc.get("textvqa-pure", np.nan)),
    gqa=format_value(language_only_llm_acc.get("gqa", np.nan)),
    wsc273=format_value(language_only_llm_acc.get("wsc273", np.nan)),
    winogrande=format_value(language_only_llm_acc.get("winogrande", np.nan)),
    arc_easy=format_value(language_only_llm_acc.get("arc_easy", np.nan)),
    arc_challenge=format_value(language_only_llm_acc.get("arc_challenge", np.nan)),
    lambada=format_value(language_only_llm_acc.get("lambada_standard", np.nan))
)

latex_code += "\\midrule\n"

for model, accuracies in table_data:
    latex_code += "{model} & {vqa_v2} & {textvqa_ocr} & {textvqa_pure} & {gqa} & {wsc273} & {winogrande} & {arc_easy} & {arc_challenge} & {lambada} \\\\\n".format(
        model=model,
        vqa_v2=format_value(accuracies["vqa-v2"]),
        textvqa_ocr=format_value(accuracies["textvqa-ocr"]),
        textvqa_pure=format_value(accuracies["textvqa-pure"]),
        gqa=format_value(accuracies["gqa"]),
        wsc273=format_value(accuracies["wsc273"]),
        winogrande=format_value(accuracies["winogrande"]),
        arc_easy=format_value(accuracies["arc_easy"]),
        arc_challenge=format_value(accuracies["arc_challenge"]),
        lambada=format_value(accuracies["lambada_standard"])
    )

latex_code += """
     \\bottomrule
    \\end{tabular}
  }
\\end{table*}
"""

print(latex_code)



\begin{table*}[h]
  \caption{\textbf{LLaVA Model Performance:} Per Dataset}
  \label{tab:per_dataset}
  \centering
  \resizebox{\linewidth}{!}{
    \begin{tabular}{l|cccccccccc}
     \toprule
     \textbf{Model} & \textbf{VQAv2} & \textbf{TextVQA OCR} & \textbf{TextVQA Pure} & \textbf{GQA} & \textbf{WSC273} & \textbf{Winogrande} & \textbf{ARC Easy} & \textbf{ARC Challenge} & \textbf{Lambada} \\
     \midrule
Original LLaVA & 30.3 & 2.4 & 3.8 & 22.2 & 53.5 & 51.9 & 39.1 & 20.1 & 11.3 \\
Language Only LLM & 0.0 & 0.0 & 0.0 & 0.0 & 56.8 & 49.6 & 44.2 & 20.0 & 23.3 \\
\midrule
LoRA & 29.0 & 1.0 & 1.7 & 18.0 & 54.6 & 49.6 & 38.9 & 21.7 & 19.5 \\
LoRA (1/2 Full Rank) & 0.1 & 0.2 & 0.1 & 0.0 & - & - & - & - & - \\
LoRA (1/2 Full Rank, Higher Alpha) & 28.7 & 1.1 & 2.7 & 19.7 & 58.6 & 50.4 & 38.7 & 18.7 & 10.1 \\
LoRA (1/2 Full Rank, RSLoRA) & 29.0 & 1.0 & 1.7 & 18.0 & 54.6 & 49.6 & 38.9 & 21.7 & 19.5 \\
LoRA (1/2 Full Rank, RSLoRA, KQV Target) & 0.0 & 0.0 & 0.0 & 0.0 & - & - & - & - & - \\
Lo